# Week 04: Staistical Inference with Python

Course-WMASDS04: Introduction to Data Science with Python
<br>Instructor: Farhana Afrin, Department of Statistics, JU

### Objectives:
- Confidence intervals for population mean
- Test of Hypothesis (t -test for mean)
***
Data source: [NHANES](https://www.cdc.gov/nchs/nhanes/index.htm) data for all the analyses below.
***

**Import required libraries**

In [2]:
# %matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
# import statsmodels.api as sm

**Read the data using the [read_csv](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) function**

In [3]:
da = pd.read_csv("nhanes_2015_2016.csv")

In [4]:
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


### Meaning of the column’s name
https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm
<br>https://wwwn.cdc.gov/nchs/nhanes/2015-2016/SMQ_I.htm

SEQN : Respondent sequence number

ALQ110 : Had at least 12 alcohol drinks/1 yr?

ALQ110: Had at least 12 alcohol drinks/lifetime?

ALQ130: How often drink alcohol over past 12 mos

SMQ020: Smoked at least 100 cigarettes in life

RIAGENDR: Gender

RIDAGEYR: Age in years at screening

RIDRETH1: Race/Hispanic origin

DMDCITZN: Citizenship status

DMDEDUC2: Education level — Adults 20+

DMDMARTL: Marital status

DMDHHSIZ: Total number of people in the Household

WTINT2YR: Full sample 2 year interview weight

SDMVPSU: Masked variance pseudo-PSU

SDMVSTRA : Masked variance pseudo-stratum

INDFMPIR: Ratio of family income to poverty

BPXSY1 : Systolic: Blood pres (1st rdg) mm Hg

BPXDI1: Diastolic: Blood pres (1st rdg) mm Hg

BPXSY2: Systolic: Blood pres (2nd rdg) mm Hg

BPXDI2 : Diastolic: Blood pres (2nd rdg) mm Hg

BMXWT: Weight (kg)

BMXHT: Standing Height (cm)

BMXBMI: Body Mass Index (kg/m**2)

BMXLEG: Upper Leg Length (cm)

BMXARML: Upper Arm Length (cm)

BMXARMC: Arm Circumference (cm)

BMXWAIST: Waist Circumference (cm)

HIQ210: Time when no insurance in past year?

### Select Variables and Drop Missing values

In [5]:
## Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI", "SMQ020"]
da = da[vars].dropna()
da.head()

,BPXSY1,RIDAGEYR,RIAGENDR,RIDRETH1,DMDEDUC2,BMXBMI,SMQ020
0,128.0,62,1,3,5.0,27.8,1
1,146.0,53,1,3,3.0,30.8,1
2,138.0,78,1,3,3.0,28.8,1
3,132.0,56,2,3,5.0,42.4,2
4,100.0,42,2,4,4.0,20.3,2


[Blood Pressure, Age in years at screening, Gender, Race/Hispanic origin, Education level — Adults 20+, Body Mass Index (kg/m**2), Smoked at least 100 cigarettes in life ]

In [6]:
da.columns

Index(['BPXSY1', 'RIDAGEYR', 'RIAGENDR', 'RIDRETH1', 'DMDEDUC2', 'BMXBMI',
       'SMQ020'],
      dtype='object')

### Rename Columns

In [7]:
da.rename(columns = {'BPXSY1':'BP',
            'RIDAGEYR': 'Age', 
            'RIAGENDR': 'Gender', 
            'RIDRETH1':'Ethnic', 
            'DMDEDUC2':'Education', 
            'BMXBMI':'BMI',
            'SMQ020':'Smoke100'}, inplace = True)
da.head()

,BP,Age,Gender,Ethnic,Education,BMI,Smoke100
0,128.0,62,1,3,5.0,27.8,1
1,146.0,53,1,3,3.0,30.8,1
2,138.0,78,1,3,3.0,28.8,1
3,132.0,56,2,3,5.0,42.4,2
4,100.0,42,2,4,4.0,20.3,2


### Create a labeled version of the gender and ethic group variables

RIAGENDRx — Gender

RIDRETH1x — Race (https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#RIAGENDR)


In [8]:
da['Gender'].unique()

array([1, 2], dtype=int64)

In [9]:
da['Ethnic'].value_counts()

3    1692
4    1058
1     896
5     771
2     685
Name: Ethnic, dtype: int64

In [10]:
da["Gender"] = da['Gender'].replace({1: "Male", 2: "Female"})
da["Ethnic"] = da['Ethnic'].replace({1: "Mexican_American", 
                                    2: "Other_Hispanic", 
                                    3: "Non_Hispanic_White",
                                    4: "Non_Hispanic_Black", 
                                    5: "Others"})
da.sample(10)

,BP,Age,Gender,Ethnic,Education,BMI,Smoke100
94,112.0,49,Male,Non_Hispanic_White,4.0,25.5,2
5538,128.0,75,Female,Non_Hispanic_White,5.0,19.9,1
5174,136.0,31,Male,Non_Hispanic_White,3.0,28.0,1
1751,156.0,63,Male,Others,2.0,23.9,2
5481,126.0,48,Female,Other_Hispanic,3.0,32.8,2
2112,124.0,56,Female,Others,3.0,33.5,2
186,126.0,45,Female,Non_Hispanic_White,5.0,33.2,2
3241,126.0,80,Female,Mexican_American,1.0,29.6,1
2907,130.0,58,Female,Non_Hispanic_Black,5.0,36.9,1
4376,110.0,26,Female,Non_Hispanic_Black,4.0,31.5,1


In [11]:
da['Smoke100'] = da['Smoke100'].replace({1:'Yes',
                                        2:'No',
                                        7:'Refused',
                                        9:'''Don't_know'''})

In [12]:
da.sample(10)

,BP,Age,Gender,Ethnic,Education,BMI,Smoke100
4715,138.0,66,Male,Other_Hispanic,2.0,32.5,Yes
1003,166.0,56,Female,Other_Hispanic,1.0,36.3,No
837,114.0,20,Female,Non_Hispanic_White,4.0,22.1,No
3749,130.0,74,Female,Mexican_American,2.0,31.5,No
883,116.0,27,Female,Non_Hispanic_White,4.0,21.1,No
4814,128.0,59,Male,Non_Hispanic_White,1.0,41.6,Yes
1772,126.0,44,Male,Non_Hispanic_Black,2.0,23.4,Yes
5578,102.0,26,Female,Mexican_American,3.0,18.7,No
5560,126.0,60,Male,Others,2.0,26.6,No
930,118.0,48,Male,Non_Hispanic_White,4.0,23.5,Yes


### Reset Index

In [13]:
da.tail()

,BP,Age,Gender,Ethnic,Education,BMI,Smoke100
5730,112.0,76,Female,Non_Hispanic_White,3.0,21.5,Yes
5731,118.0,26,Male,Non_Hispanic_White,5.0,33.8,No
5732,154.0,80,Female,Non_Hispanic_White,4.0,31.0,Yes
5733,104.0,35,Male,Non_Hispanic_White,1.0,26.0,Yes
5734,118.0,24,Female,Non_Hispanic_White,5.0,21.4,No


In [14]:
da.shape

(5102, 7)

In [15]:
da.size

35714

In [16]:
da.reset_index(inplace = True)

In [17]:
# drop 'index' column, as it is not important
# da.drop('index', axis = 1, inplace = True)
da.tail()

,index,BP,Age,Gender,Ethnic,Education,BMI,Smoke100
5097,5730,112.0,76,Female,Non_Hispanic_White,3.0,21.5,Yes
5098,5731,118.0,26,Male,Non_Hispanic_White,5.0,33.8,No
5099,5732,154.0,80,Female,Non_Hispanic_White,4.0,31.0,Yes
5100,5733,104.0,35,Male,Non_Hispanic_White,1.0,26.0,Yes
5101,5734,118.0,24,Female,Non_Hispanic_White,5.0,21.4,No


## Confidence intervals for the mean

- Calculate the mean BMI for all women and for all men
- Calculate the standard deviation 
- Calculate the upper limit and lower limit

##### Mean BMI for both male and female

In [18]:
da.groupby("Gender").agg({"BMI": np.mean}).reset_index()

,Gender,BMI
0,Female,30.049129
1,Male,28.925630


##### Standard deviation of the BMI data for both male and female

In [19]:
da.groupby('Gender', as_index = False).agg({'BMI':['count','mean','std']})

Gender   BMI                     
          count       mean       std
0  Female  2640  30.049129  7.647669
1    Male  2462  28.925630  6.128041

In [20]:
# da.groupby("RIAGENDR").agg({"BMXBMI": [np.mean, np.std, np.size]})

##### standard error of the mean BMI for women and for men:

In [21]:
sem_female = 7.648 / np.sqrt(2976)
sem_male = 6.128 / np.sqrt(2759)
print(sem_female, sem_male)

0.14019464196041376 0.11666562349591446


In [22]:
sem_female = 7.753 / np.sqrt(2976)
sem_male = 6.253 / np.sqrt(2759)
print(sem_female, sem_male)

0.14211938534506902 0.119045388988243


### The 95% confidence interval for female BMI is thus calculated as follows:

In [23]:
lcb_female = 29.94 - 1.96 * 7.753 / np.sqrt(2976)
ucb_female = 29.94 + 1.96 * 7.753 / np.sqrt(2976)
print(lcb_female, ucb_female)

29.661446004723665 30.218553995276338


### 95% Confidence interval for female BMI using Statsmodels:

In [24]:
import statsmodels.api as sm
female_bmi = da.loc[da.Gender=="Female", "BMI"].dropna()
sm.stats.DescrStatsW(female_bmi).zconfint_mean()

(29.757402734652434, 30.340854841105152)

In [30]:
sm.stats.DescrStatsW(female_bmi).tconfint_mean(alpha = 0.05)

(29.75726887555235, 30.340988700205237)

### 95% Confidence interval for female BMI using *t.interval* from scipy

In [29]:
from scipy.stats import t
t.interval(.95, len(female_bmi)-1, loc=np.mean(female_bmi), scale=np.std(female_bmi)/np.sqrt(len(female_bmi)-1))

(29.757268875552402, 30.34098870020529)

# Test of Hypothesis

**Research Question:** 
let’s test whether the mean systolic blood pressure (BP) in the NHANES dataset (averaged over the three measurements that were taken for each person) is greater than 120 mm, which is the standard value for normal systolic BP.
<br> https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Statistical_Thinking_for_the_21st_Century_(Poldrack)/29%3A_Comparing_Means_in_R/29.01%3A_Testing_the_Value_of_a_Single_Mean_(Section_28.1)

In [26]:
da['BP'].mean()

125.62681301450412

In [24]:
# d1 = sm.stats.DescrStatsW(female_bmi)
# tstat, pval, df = d1.ttest_mean([0])
# tstat; pval; df
# print(tstat, pval, df)

***
***